In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "tr", split="train")
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "tr", split="test")


Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 25998it [00:00, 296422.64it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 10127it [00:00, 311232.13it/s]


Generating test split: 0 examples [00:00, ? examples/s]


Reading metadata...: 10143it [00:00, 316312.94it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 151it [00:00, 227492.78it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 3593it [00:00, 305428.23it/s]


In [2]:
print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 25998
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 10143
    })
})


In [3]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 25998
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 10143
    })
})


In [4]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")

In [5]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="azerbaijani", task="transcribe")

In [6]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="azerbaijani", task="transcribe")

In [7]:
print(common_voice["train"][0])

{'audio': {'path': '/home/user/.cache/huggingface/datasets/downloads/extracted/683c6f874760e64ca0a4107d56d97ef171e5dd8fdbcaf9ac15b5f3df0e085d89/tr_train_0/common_voice_tr_28856093.mp3', 'array': array([ 1.42108547e-14, -7.10542736e-14, -1.98951966e-13, ...,
       -9.25481118e-07,  6.68777466e-06,  5.80005053e-06], shape=(157248,)), 'sampling_rate': 48000}, 'sentence': 'Sadece birkaç gün.'}


In [8]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [31]:
print(common_voice["train"][0])

{'audio': {'path': '/home/user/.cache/huggingface/datasets/downloads/extracted/683c6f874760e64ca0a4107d56d97ef171e5dd8fdbcaf9ac15b5f3df0e085d89/tr_train_0/common_voice_tr_28856093.mp3', 'array': array([-1.45519152e-11, -7.27595761e-12, -1.81898940e-12, ...,
        1.13121732e-05, -1.24729513e-06, -1.83326620e-06], shape=(52416,)), 'sampling_rate': 16000}, 'sentence': 'Sadece birkaç gün.'}


In [10]:
print(common_voice["train"][1])

{'audio': {'path': '/home/user/.cache/huggingface/datasets/downloads/extracted/683c6f874760e64ca0a4107d56d97ef171e5dd8fdbcaf9ac15b5f3df0e085d89/tr_train_0/common_voice_tr_28856095.mp3', 'array': array([ 4.54747351e-12, -1.81898940e-12, -5.45696821e-12, ...,
        1.26162095e-05,  3.88094231e-05,  8.69208179e-06], shape=(58176,)), 'sampling_rate': 16000}, 'sentence': 'Çok kolaydır.'}


In [11]:
print(common_voice["train"][0]['sentence'])

Sadece birkaç gün.


In [12]:
tokenizer(common_voice["train"][0]['sentence']).input_ids

[50258, 50304, 50359, 50363, 50, 25978, 1904, 2330, 1138, 14472, 13, 50257]

In [13]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [14]:
common_voice_pre = common_voice

In [15]:
common_voice_pre = common_voice_pre.map(prepare_dataset, remove_columns=common_voice_pre.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/25998 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/10143 [00:00<?, ? examples/s]

In [16]:
common_voice_pre

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 25998
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 10143
    })
})

In [17]:
common_voice_pre['train']

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 25998
})

In [18]:
common_voice_pre['train']['labels'][0]

[50258, 50304, 50359, 50363, 50, 25978, 1904, 2330, 1138, 14472, 13, 50257]

In [19]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

In [20]:
model.generation_config.language = "azerbaijani"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [21]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [22]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [23]:
import evaluate

metric = evaluate.load("wer")

In [24]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [26]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir='./whisper-tiny-az',
    per_device_train_batch_size=32,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,  # Already enabled to reduce memory usage
    gradient_checkpointing_kwargs={'use_reentrant': False},  # Add this to avoid graph issues
    fp16=torch.cuda.is_available(),
    eval_strategy='steps',
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=['tensorboard'],
    load_best_model_at_end=True,
    metric_for_best_model='wer',
    greater_is_better=False,
    push_to_hub=False,
)

In [27]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice_pre["train"],
    eval_dataset=common_voice_pre["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/tmp/ipykernel_26274/1369350931.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [28]:
processor.save_pretrained(training_args.output_dir)

[]

In [29]:
trainer.train()

You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
1000,0.261800,0.648022,49.240263
2000,0.177100,0.661262,48.889018
3000,0.108100,0.686802,49.674953
4000,0.092800,0.698702,49.475073


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/opt/venv/lib/python3.11/site-packages/transformers/modeling_utils.py:3852: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warni

TrainOutput(global_step=4000, training_loss=0.24152549946308136, metrics={'train_runtime': 29830.3658, 'train_samples_per_second': 8.582, 'train_steps_per_second': 0.134, 'total_flos': 6.291356092416e+18, 'train_loss': 0.24152549946308136, 'epoch': 9.829028290282903})

In [30]:
# Save the fine-tuned model
model.save_pretrained('./whisper-tiny-az')
processor.save_pretrained('./whisper-tiny-az')

[]

In [35]:
import torchaudio

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def transcribe_audio(audio_path, target_sampling_rate=16000):
    # Load audio
    audio, sr = torchaudio.load(audio_path)
    
    # Resample audio to 16000 Hz if necessary
    if sr != target_sampling_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sampling_rate)
        audio = resampler(audio)
    
    # Ensure audio is mono (Whisper expects single-channel audio)
    if audio.shape[0] > 1:
        audio = torch.mean(audio, dim=0, keepdim=True)
    
    # Process audio for Whisper model
    input_features = processor(audio[0].numpy(), sampling_rate=target_sampling_rate, return_tensors='pt').input_features.to(device)
    
    # Generate transcription
    predicted_ids = model.generate(input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    
    return transcription

# Example usage
file = "/home/user/.cache/huggingface/datasets/downloads/extracted/683c6f874760e64ca0a4107d56d97ef171e5dd8fdbcaf9ac15b5f3df0e085d89/tr_train_0/common_voice_tr_28856093.mp3"
file2 = "/home/user/.cache/huggingface/datasets/downloads/extracted/683c6f874760e64ca0a4107d56d97ef171e5dd8fdbcaf9ac15b5f3df0e085d89/tr_train_0/common_voice_tr_28856095.mp3"

print('Transcription:', transcribe_audio(file))
print('Transcription:', transcribe_audio(file2))

Transcription: Sadece birkaç gün.
Transcription: Çok kolaydır.
